In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [2]:
df = pd.read_excel("../data/sods.xlsx")

In [3]:
df.head(5)

,idade,genero,etnia,pcd,vive_no_brasil,estado_moradia,nivel_ensino,formacao,tempo_experiencia_dados,linguagens_preferidas,bancos_de_dados,cloud_preferida,cargo
0,39.0,Masculino,Parda,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 1 a 2 anos,R,"PostgreSQL, Oracle, MySQL, SQL SERVER",Amazon Web Services (AWS),DBA/Administrador de Banco de Dados
1,32.0,Masculino,Parda,Não,True,Pará (PA),Graduação/Bacharelado,Outras Engenharias,de 3 a 4 anos,NaN,NaN,NaN,NaN
2,53.0,Masculino,Branca,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 3 a 4 anos,Python,"PostgreSQL, MySQL, Oracle, DB2",Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...
3,27.0,Masculino,Branca,Não,True,Minas Gerais (MG),Doutorado ou Phd,Estatística/ Matemática / Matemática Computaci...,de 4 a 6 anos,Python,Google BigQuery,Não sei opinar,Cientista de Dados/Data Scientist
4,46.0,Feminino,Branca,Não,True,Pará (PA),Pós-graduação,Computação / Engenharia de Software / Sistemas...,Não tenho experiência na área de dados,Python,Microsoft Access,Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...


## Extraindo informações sobre o dataframe

In [4]:
df.describe()

,idade
count,4197.000000
mean,31.169168
std,6.902803
min,18.000000
25%,26.000000
50%,30.000000
75%,35.000000
max,54.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4271 entries, 0 to 4270
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   idade                    4197 non-null   float64
 1   genero                   4262 non-null   object 
 2   etnia                    4271 non-null   object 
 3   pcd                      4271 non-null   object 
 4   vive_no_brasil           4271 non-null   bool   
 5   estado_moradia           4156 non-null   object 
 6   nivel_ensino             4271 non-null   object 
 7   formacao                 4149 non-null   object 
 8   tempo_experiencia_dados  3694 non-null   object 
 9   linguagens_preferidas    2979 non-null   object 
 10  bancos_de_dados          2978 non-null   object 
 11  cloud_preferida          2981 non-null   object 
 12  cargo                    2981 non-null   object 
dtypes: bool(1), float64(1), object(11)
memory usage: 404.7+ KB


In [6]:
df.isnull().sum()

idade                        74
genero                        9
etnia                         0
pcd                           0
vive_no_brasil                0
estado_moradia              115
nivel_ensino                  0
formacao                    122
tempo_experiencia_dados     577
linguagens_preferidas      1292
bancos_de_dados            1293
cloud_preferida            1290
cargo                      1290
dtype: int64

## Iniciando os Tratamentos com os dados nulos

substituindo os valores NaN das colunas por 'Prefiro não informar'

In [7]:
df.loc[:,'genero'] = df['genero'].fillna('Prefiro não informar')

In [8]:
df.loc[:,'estado_moradia'] = df['estado_moradia'].fillna('Não Informado')

In [9]:
df.loc[:, 'formacao'] = df['formacao'].fillna('Prefiro não informar')

#### Analisando a idade media geral por genero
Para substituir os valores nulos pela idade media por genero

In [10]:
df['idade'].mean()

np.float64(31.169168453657374)

In [11]:
df.groupby(['genero'])['idade'].mean()

genero
Feminino                31.318052
Masculino               31.147876
Prefiro não informar    26.473684
Name: idade, dtype: float64

#### Realizando essa substituição dos valores nulos pela idade media por genero

In [12]:
idades_generos = df.groupby(['genero'])['idade'].mean()

In [13]:
f = round(idades_generos['Feminino'])
m = round(idades_generos['Masculino'])
n = round(idades_generos['Prefiro não informar'])

print (f,m,n)

31 31 26


In [14]:
condicao = (df['idade'].isna()) & (df['genero'] == 'Masculino')
df.loc[condicao, 'idade'] = m

condicao = (df['idade'].isna()) & (df['genero'] == 'Prefiro não informar')
df.loc[condicao, 'idade'] = n

condicao = (df['idade'].isna()) & (df['genero'] == 'Feminino')
df.loc[condicao, 'idade'] = f

Verificando quantos dados são nulos novamente

In [15]:
df.isnull().sum()

idade                         0
genero                        0
etnia                         0
pcd                           0
vive_no_brasil                0
estado_moradia                0
nivel_ensino                  0
formacao                      0
tempo_experiencia_dados     577
linguagens_preferidas      1292
bancos_de_dados            1293
cloud_preferida            1290
cargo                      1290
dtype: int64

 #### Verificando possiveis linhas com diversas dados faltosos, importantes, devido ao grande e semelhante numero de dados nulos nas colunas "linguagem_preferidas": 1292, "banco_de_dados": 1293, "cloud_preferida": 1290, "cargo": 1290. Amazenando essa correlação e uma coluna pra ver se todos essas colunas são nulas nas mesmas linhas 

In [16]:
df['carreira_nan'] = df['cargo'].isna() & df['cloud_preferida'].isna() & df['bancos_de_dados'].isna() & df['linguagens_preferidas'].isna()

#### Devido ao uso da função sum() usada nessa coluna nova é possivel verificar que ela retornou True 1290 vezes oque concidente com o intervalo de dados nulos das colunas

In [17]:
df['carreira_nan'].sum()

np.int64(1290)

#### Verificando agora para as colunas "banco_de_dados": 1293, "linguagens_preferidas": 1292 (dados nulos)

In [18]:
df['linguagem_banco'] = df['bancos_de_dados'].isna() & df['linguagens_preferidas'].isna()

df['linguagem_banco'].sum()

np.int64(1292)

#### Analisando se existe uma relação, se eles são nulos na mesma linha, entre entre os dados das colunas "tempo_experiencia_dados" e "formacao"
Não foi valido essa relação

In [19]:
df['tempo_formacao'] = df['tempo_experiencia_dados'].isna() & df['formacao'].isna()

df['tempo_formacao'].sum()

np.int64(0)

#### Excluindo todas as linhas que possuem 'carreira_nan' como True. Pois essas linhas possuem 4 dados vazios importantes e que possivelmente iriam influenciar o modelo preditivo 

In [20]:
df = df[~df['carreira_nan']]

Verificando os dados vazios

In [21]:
df.isnull().sum()

idade                      0
genero                     0
etnia                      0
pcd                        0
vive_no_brasil             0
estado_moradia             0
nivel_ensino               0
formacao                   0
tempo_experiencia_dados    0
linguagens_preferidas      2
bancos_de_dados            3
cloud_preferida            0
cargo                      0
carreira_nan               0
linguagem_banco            0
tempo_formacao             0
dtype: int64

Substituindo os valores vazios de "formacao" por 'Prefiro não informar' e "linguagens_preferidas" por 'Não sei opinar'

In [22]:
df.loc[:,'linguagens_preferidas'] = df['linguagens_preferidas'].fillna('Não sei opinar')

In [23]:
df.loc[:,'bancos_de_dados'] = df['bancos_de_dados'].fillna('Prefiro não informar')

In [24]:
df.isnull().sum()

idade                      0
genero                     0
etnia                      0
pcd                        0
vive_no_brasil             0
estado_moradia             0
nivel_ensino               0
formacao                   0
tempo_experiencia_dados    0
linguagens_preferidas      0
bancos_de_dados            0
cloud_preferida            0
cargo                      0
carreira_nan               0
linguagem_banco            0
tempo_formacao             0
dtype: int64

In [25]:
df.head(5)

,idade,genero,etnia,pcd,vive_no_brasil,estado_moradia,nivel_ensino,formacao,tempo_experiencia_dados,linguagens_preferidas,bancos_de_dados,cloud_preferida,cargo,carreira_nan,linguagem_banco,tempo_formacao
0,39.0,Masculino,Parda,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 1 a 2 anos,R,"PostgreSQL, Oracle, MySQL, SQL SERVER",Amazon Web Services (AWS),DBA/Administrador de Banco de Dados,False,False,False
2,53.0,Masculino,Branca,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 3 a 4 anos,Python,"PostgreSQL, MySQL, Oracle, DB2",Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...,False,False,False
3,27.0,Masculino,Branca,Não,True,Minas Gerais (MG),Doutorado ou Phd,Estatística/ Matemática / Matemática Computaci...,de 4 a 6 anos,Python,Google BigQuery,Não sei opinar,Cientista de Dados/Data Scientist,False,False,False
4,46.0,Feminino,Branca,Não,True,Pará (PA),Pós-graduação,Computação / Engenharia de Software / Sistemas...,Não tenho experiência na área de dados,Python,Microsoft Access,Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...,False,False,False
5,32.0,Feminino,Parda,Não,True,Ceará (CE),Pós-graduação,Ciências Biológicas/ Farmácia/ Medicina/ Área ...,Não tenho experiência na área de dados,Python,Google BigQuery,Google Cloud (GCP),Professor,False,False,False


#### Excluindo as colunas que foram usadas para analisar as correlações

In [26]:
df = df.drop(columns='tempo_formacao')
df = df.drop(columns='carreira_nan')
df = df.drop(columns='linguagem_banco')

#### Analisando as linguagens preferidas por genero. Verificando que existem respostas inconsistentes  

In [27]:
df.groupby(['genero','linguagens_preferidas']).size()

genero                linguagens_preferidas           
Feminino              C/C++/C#                               7
                      Dax                                    1
                      Excel                                  1
                      Java                                   1
                      Julia                                  4
                      KQL                                    1
                      Nenhum                                 1
                      NÃO SEI                                1
                      Não sei                                1
                      Não sei opinar                         1
                      Não uso                                1
                      Não utilizo                            1
                      PySpark                                2
                      Python                               617
                      R                                     83


#### Tratamento dos dados da coluna "linguagens_preferidas"

In [28]:
df['linguagens_preferidas'] = df['linguagens_preferidas'].apply(lambda x: x.lower())

In [29]:
df['linguagens_preferidas'] = df['linguagens_preferidas'].replace("não sei", "não sei opinar")

df.loc[df['linguagens_preferidas'] == "não uso", 'linguagens_preferidas'] = "não sei opinar"

df.loc[df['linguagens_preferidas'] == "não utilizo", 'linguagens_preferidas'] = "não sei opinar"

df.loc[df['linguagens_preferidas'] == "nenhum", 'linguagens_preferidas'] = "não sei opinar"

df.loc[df['linguagens_preferidas'] == "nao atuo com programação", 'linguagens_preferidas'] = "não sei opinar"

df.loc[df['linguagens_preferidas'] == "nenhuma", 'linguagens_preferidas'] = "não sei opinar"

df['linguagens_preferidas'] = df['linguagens_preferidas'].replace("aql", "sql")

df['linguagens_preferidas'] = df['linguagens_preferidas'].replace("kql", "sql")

df['linguagens_preferidas'] = df['linguagens_preferidas'].replace("m", "m language")

df['linguagens_preferidas'] = df['linguagens_preferidas'].replace("xlsx", "excel")

df['linguagens_preferidas'] = df['linguagens_preferidas'].replace("softwares estatísticos como spss", "não sei opinar")

In [30]:
df.groupby(['genero','linguagens_preferidas']).size()

genero                linguagens_preferidas
Feminino              c/c++/c#                    7
                      dax                         1
                      excel                       1
                      java                        1
                      javascript                  1
                      julia                       4
                      não sei opinar              7
                      pyspark                     2
                      python                    617
                      r                          83
                      scala                       8
                      spark                       2
                      sql                        19
                      sql postegres               1
Masculino             c/c++/c#                   18
                      clojure                     1
                      elixir                      3
                      excel                       2
                    

#### Tratamento dos Dados da coluna 'bancos_de_dados'

In [31]:
df['bancos_de_dados'] = df['bancos_de_dados'].apply(lambda x: x.lower())
filtro = df['bancos_de_dados'].str.lower().str.startswith('n', na=False)
valores_unicos_com_n = df.loc[filtro, 'bancos_de_dados'].unique()

valores_unicos_com_n

array(['nenhum', 'não utilizo.', 'não atuo ainda nisso',
       'neo4j, mongodb, snowflake, amazon aurora ou rds, mysql, elaticsearch, sqlite, mariadb, hbase, hive, sql server, dynamodb, oracle, cassandra, s3, presto, amazon redshift, databricks, amazon athena',
       'nenhuma', 'não utiluzo', 'não temos bd', 'não uso nenhum',
       'nao utilizo', 'nsa', 'não utilizo nenhum',
       'nenhuma das respostas', 'nenhum dos listados, apenas excel',
       'não reconheço nenhum',
       'não atuo na área ainda, estou em transição de carreira',
       'nao uso', 'nenhum desses', 'não uso', 'neo4j, mysql, postgresql',
       'não trabalho com banco diretamente', 'não utilizo',
       'não possuímos um banco de dados.',
       'não utilizamos banco de dados', 'nenhum acima',
       'não atuo na área ainda', 'não trabalho com base de dados.',
       'não uso bd no trabalho, somente em cursos',
       'nenhum dos citados acima', 'nosso banco é no excel', 'não usamos',
       'neo4j, s3, mongodb

#### Utilizando regex para substituir todos os dados que poderiam ser 'não', 'nao', 'nenhum', 'nenhuma', 'nda', 'não aplicavel', 'none'

In [32]:
padrao_regex = r'(?i)^(n[aã]o|nenhum|nenhuma|nda|n/a|none).*'
df['bancos_de_dados'] = df['bancos_de_dados'].replace("nd", "não uso")

df['bancos_de_dados'] = df['bancos_de_dados'].replace(
    to_replace=padrao_regex,
    value='não uso',
    regex=True
)

In [33]:
filtro = df['bancos_de_dados'].str.lower().str.startswith('n', na=False)
valores_unicos_com_n = df.loc[filtro, 'bancos_de_dados'].unique()

valores_unicos_com_n

array(['não uso',
       'neo4j, mongodb, snowflake, amazon aurora ou rds, mysql, elaticsearch, sqlite, mariadb, hbase, hive, sql server, dynamodb, oracle, cassandra, s3, presto, amazon redshift, databricks, amazon athena',
       'nsa', 'neo4j, mysql, postgresql', 'nosso banco é no excel',
       'neo4j, s3, mongodb, google bigquery',
       'neo4j, google bigquery, postgresql, google firestore, firebase, amazon athena'],
      dtype=object)

Resentando Index 

In [34]:
df = df.reset_index(drop=True)

In [35]:
df['bancos_de_dados'].unique()

array(['postgresql, oracle, mysql, sql server',
       'postgresql, mysql, oracle, db2', 'google bigquery', ...,
       'google bigquery, postgresql, mysql',
       'postgresql, oracle, mariadb, hbase, mysql',
       'oracle, firebird, sql server'], shape=(1429,), dtype=object)

In [36]:
df.head(5)

,idade,genero,etnia,pcd,vive_no_brasil,estado_moradia,nivel_ensino,formacao,tempo_experiencia_dados,linguagens_preferidas,bancos_de_dados,cloud_preferida,cargo
0,39.0,Masculino,Parda,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 1 a 2 anos,r,"postgresql, oracle, mysql, sql server",Amazon Web Services (AWS),DBA/Administrador de Banco de Dados
1,53.0,Masculino,Branca,Não,True,Distrito Federal (DF),Pós-graduação,Computação / Engenharia de Software / Sistemas...,de 3 a 4 anos,python,"postgresql, mysql, oracle, db2",Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...
2,27.0,Masculino,Branca,Não,True,Minas Gerais (MG),Doutorado ou Phd,Estatística/ Matemática / Matemática Computaci...,de 4 a 6 anos,python,google bigquery,Não sei opinar,Cientista de Dados/Data Scientist
3,46.0,Feminino,Branca,Não,True,Pará (PA),Pós-graduação,Computação / Engenharia de Software / Sistemas...,Não tenho experiência na área de dados,python,microsoft access,Amazon Web Services (AWS),Desenvolvedor/ Engenheiro de Software/ Analist...
4,32.0,Feminino,Parda,Não,True,Ceará (CE),Pós-graduação,Ciências Biológicas/ Farmácia/ Medicina/ Área ...,Não tenho experiência na área de dados,python,google bigquery,Google Cloud (GCP),Professor


In [37]:
df.to_excel('../data/sods_limpo.xlsx', index=False)